# Synthetic control (lite) for campaigns

This notebook demonstrates a **synthetic control style** estimator.

Practical modes:
1) covariates-only (regression on controls)
2) donor-series mode (if you have multiple untreated units)

In v1 we implement a ridge-based 'lite' estimator.


In [ ]:
import sys
from pathlib import Path

# Ensure `src/` is importable when running from repo root
repo_root = Path.cwd()
src_path = repo_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tecore.causal import DataSpec, ImpactConfig, ImpactMethod, run_impact


## 1) Load example dataset

In [ ]:
df = pd.read_csv(repo_root / "data" / "example_ts" / "example_daily.csv")
df.head()

## 2) Synthetic control (covariates-only)

In [ ]:
spec = DataSpec(
    date_col="date",
    y_col="y",
    x_cols=["sessions", "active_users", "marketing_spend", "external_index"],
    freq="D",
    add_time_trend=True,
    add_day_of_week=True,
)

cfg = ImpactConfig(
    intervention_date="2025-05-01",
    method=ImpactMethod.SYNTHETIC_CONTROL,
    ridge_alpha=1.0,
    bootstrap_iters=200,
    block_size=7,
    alpha=0.05,
    run_placebo=True,
    n_placebos=25,
)

res = run_impact(df, spec, cfg)
res.summary()

### Plot observed vs counterfactual

In [ ]:
eff = res.effect_series.copy()
eff["date"] = pd.to_datetime(eff["date"])

plt.figure()
plt.plot(eff["date"], eff["y"], label="observed")
plt.plot(eff["date"], eff["y_hat"], label="counterfactual")
plt.axvline(pd.Timestamp(cfg.intervention_date), linestyle="--")
plt.title("Synthetic control (lite): observed vs counterfactual")
plt.legend()
plt.tight_layout()
plt.show()

## 3) Donor-series mode (concept)

If you have donor time series (untreated geos/segments/platforms), pass them as `x_cols`.
Example:
- y: treated geo metric
- x: donor geo metrics

In this notebook we only demonstrate the covariate-only mode using the bundled dataset.


## 4) Save outputs

In [ ]:
out_dir = repo_root / "out" / "notebook_10_synth_control"
out_dir.mkdir(parents=True, exist_ok=True)

eff_out = out_dir / "effect_series.csv"
eff_to_save = res.effect_series.copy()
eff_to_save["date"] = pd.to_datetime(eff_to_save["date"]).dt.strftime("%Y-%m-%d")
eff_to_save.to_csv(eff_out, index=False)

import json
with open(out_dir / "summary.json", "w", encoding="utf-8") as f:
    json.dump(res.summary(), f, indent=2, ensure_ascii=False)

print("Wrote:", out_dir)